In [2]:
# Importing the libraries 
import pandas as pd
import numpy as np
import random
import pickle
import re
import regex  # for better, more capbale regex api
import os
import zipfile
import more_itertools
from itertools import chain
import datetime
import time
from statsmodels.stats.proportion import proportion_confint
# active labeler related
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import ComplementNB  # corrects for class imbalance, SGD is pretty good too
from sklearn.pipeline import Pipeline
from superintendent import ClassLabeller
from IPython.display import display, Markdown

pipeline = Pipeline([
    ('vect', CountVectorizer(analyzer='char', ngram_range=(1,2))),
    ('tfidf', TfidfTransformer()),
    ('clf', ComplementNB()),
])
print('done')

done


In [1]:
# Set up columns to keep, fields, locations for writing
rootpath = "/hdd/work/d4ad_standardization/"
processedpath = "D4AD_Standardization/data/processed/"
externalpath = "D4AD_Standardization/data/external/"
interimpath = "D4AD_Standardization/data/interim/"

content_is = "standardized_name_and_name1"


print('done')

done


In [3]:

filepath = "standardized_name_and_name1.csv" # builds off of notebook 5 work

columns = [
    "STANDARDIZEDNAME_1",
    "STANDARDIZEDNAME",
    "DESCRIPTION",
    "FEATURESDESCRIPTION",
    "NAME_1",
    "NAME",
    "PREREQUISITES",
    "STREET1",
    "CITY",
    "STATE",
    "ZIP",
    "WEBSITE",
    "COUNTY",
    "NONGOVAPPROVAL",
    "STATECOMMENTS",
    "CIPCODE",
    "PROVIDERID",
    "APPROVINGAGENCYID"
]

columns_to_save = ['STANDARDIZED_DESCRIPTION', 'STANDARDIZED_FEATURESDESCRIPTION'] + columns

SKIP_THIS = True # helps me be able to run all and not worry about pulling things
# I already know I have on disk

#df = pd.read_excel(rootpath + interimpath + filepath, usecols=columns)
df = pd.read_csv(rootpath + interimpath + filepath, usecols=columns)
print('done')

done


In [4]:
pd.set_option('display.max_rows', False)

the_df = df #df.sample(n=10000, random_state=42)

In [5]:
# 2) Here we apply the abbreviation expansion to the
# description columns. This code is repeated from the 5.0 notebook and should be externalized into ./src somewhere
#
# We first construct the abbreviation mapper
#
# We also store off a copy of the df for manipulation
# this has older name fields, for informing on funding (WOIA) and degree type (?)
# as well as the standardized fields so taht we can remove the extranous content still in it
# Note: this is a conflatation of responsibilites 

label_mapper = pd.read_csv(
    rootpath + externalpath + "label_mapper.csv"
)

draft_output = the_df[['DESCRIPTION', 'FEATURESDESCRIPTION',
                       'STANDARDIZEDNAME_1', 'STANDARDIZEDNAME',
                       'NAME_1', 'NAME']]


def make_term_grouped_regex(term="", right_regex="", left_regex=""):
    mystr = left_regex + '(' +\
                re.escape(term) +\
            ')' + right_regex
    return mystr

def make_grouped_regexes(replacement, left_regex="", right_regex=""):
    return (make_term_grouped_regex(left_regex=left_regex,
                                    term=key,
                                    right_regex=right_regex)\
            for key in replacement.keys()
    )

def construct_map(label_mapper=label_mapper):
    return {
        **dict(zip(label_mapper.abbreviation, label_mapper.expanded))
    }

replacement_map = construct_map()

abbrevation_pattern =\
    regex.compile(
        "(?p)" +
        "|".join(   # match words at start of string
            make_grouped_regexes(replacement_map, left_regex=r'^', right_regex=r'[\s:]')
        ) + "|" +\
        "|".join(   # match words surrounded by spaces
            make_grouped_regexes(replacement_map, left_regex=r'\s', right_regex=r'\s')
        ) + "|" +\
        "|".join(   # match words that make up entire fields, e.g. 'Nursing'
            make_grouped_regexes(replacement_map, left_regex=r'^', right_regex=r'$')
        ) + "|" +\
        "|".join(   # match words at end of string preceded by space or slash
            make_grouped_regexes(replacement_map, left_regex=r'[\s/]', right_regex=r'$')
        ) + "|" +\
        "|".join(   # match words within string that follow a slash, end with a space or slash
            make_grouped_regexes(replacement_map, left_regex=r'/', right_regex=r'[\s/]')
        )
    )

def multiple_mapper(string):
    return abbrevation_pattern.sub(
        lambda x: \
        x.group().replace( # replace the found string
            more_itertools.first_true(x.groups() # where the first matched group...
        ),  replacement_map[more_itertools.first_true(x.groups())] # ... is replaced with the lookup
    ), string)
print('done1')

done1


In [12]:
# ... with the abbreviation mapper in hand we now simply apply to both description columns
# it takes about 2 minutes each to run through all rows.abs
start = datetime.datetime.now()

draft_output['STANDARDIZED_DESCRIPTION'] =\
    draft_output['DESCRIPTION'].dropna().map(multiple_mapper)
draft_output['STANDARDIZED_FEATURESDESCRIPTION'] =\
    draft_output['FEATURESDESCRIPTION'].dropna().map(multiple_mapper)

end = datetime.datetime.now()
print(f"Done! That took {(end-start)} time")

KeyError: 'FEATUREsDESCRIPTION'

That took 0:00:02.002295 time


In [104]:
# 3) 
# Then go after odd static patterns that are common 
# ... A.A., AAS,e ends-with "/", etc etc
# "Applied Certificate in..." <--- thing is, this could really be a program
# the_df.STANDARDIZEDNAME_1 =\
#     the_df.STANDARDIZEDNAME_1.str.replace("A.A.","", case=False)

#  I think the longest matching tends to account for this when known
# things like Oracle OCP are there

18

In [88]:
# This is the evaluation part of the program and course name standardizations
# along with the provider name. My goal is to have 85%+ standardized, send out
# that 85% will come from the jefferey's interval

# Evaluation Rubric:
#   A) Here we label clearly wrong snippets, anything that is marginal we mark as
# standardized for purposes of this evaluation because we want to err on the side
# of giving overly specific information, which includes odd info
#   B) We also click through quickly, not overly dwelling one any one example, the
# goal here is to get the evaulation done quickly since it's so manual
#   C) For now we ignore casingl there does need to be a camel casing applied to
# all caps

# We create a series of data to evaluate
columns_to_check = ['MULTI_REPLACE_STANDARDIZEDNAME_1'] # we know NAME is mostly fine, 'STANDARDIZEDNAME']
the_data =\
    np.concatenate(
        (
            draft_output[columns_to_check[0]].to_numpy(),
            #the_df[columns_to_check[1]].to_numpy()
        )
    )
    
# we shuffle the data to elminate any bias across/within the columns when
# evaluting
random.Random(42).shuffle(the_data)
print('done', f'The data is {len(the_data)} long')

done The data is 100 long


In [89]:
def display_func(row):
    """
    The display function gets passed your data - in the
    case of a dataframe, it gets passed a row - and then
    has to "display" your data in whatever way you want.

    It doesn't need to return anything
    """
    display(Markdown(row))
    #display(Markdown("**At:** " + row["timestamp"]))

def preprocessor(x, y):
    # only take standardized column, leave everything else
    return x, y

verification_widget = ClassLabeller(
    features=the_data,
    model=pipeline,
    model_preprocess=preprocessor,
    display_func=display_func,
    options=['standardized', 'not standardized'],
    acquisition_function='margin'
)

verification_widget

ClassLabeller(children=(HBox(children=(HBox(children=(FloatProgress(value=0.0, description='Progress:', max=1.…

In [92]:
# insert bionomial proprtion esimator here

def print_CI(labels, response_is_standardized = "standardized", method = "jeffreys"):
    successful_count = sum(
        response_is_standardized == label for label in labels
    )
    not_examined_count = sum(
        None == label for label in labels
    )

    CI = proportion_confint(
            count= successful_count,
            nobs= len(labels) - not_examined_count,
            alpha = 0.95,
            method=method
        )
    print(f"{method} bionomial proportion is: [{CI[0]:.2f}, {CI[1]:.2f}]",
)
    print(f"We examined {len(labels) - not_examined_count} labels, of which {successful_count} are correct. There are {len(labels)} labels.")
print_CI(labels=verification_widget.new_labels)


jeffreys bionomial proportion is: [0.95, 0.95]
We examined 100 labels, of which 95 are correct. There are 100 labels.


In [171]:
# 4)
# Now we write out the verfiied results
# ... finally we can write this out as our first complete lookup table
# for the NAME field
write_out = the_df

write_out['STANDARDIZEDNAME_1'] =\
    draft_output['MULTI_REPLACE_STANDARDIZEDNAME_1']

# shuffe the rows to better remove temporal baises
write_out =\
    the_df.sample(frac=1, random_state=42, axis=0).reset_index(drop=True)

write_out.to_csv(rootpath + interimpath + content_is + ".csv",
                index = False,
                chunksize = 10000,
                columns=columns_to_save)

write_out.to_excel(rootpath + processedpath + content_is + ".xls",
            sheet_name="Standardized NAME and NAME_1",
            index=False,
            columns=columns_to_save)
print('done')

done
